In [4]:
import joblib as jb
from pathlib import Path
import pandas as pd
model_dir = Path("../models")
model = jb.load(model_dir/"final_rf_tuned.joblib")
features = jb.load(model_dir /"feature_list.joblib")
preds_saved = jb.load(model_dir / "final_pred_rf_tuned.joblib")

X_train = jb.load(model_dir / "X_train.joblib")
X_test = jb.load(model_dir / "X_test.joblib")
Y_train = jb.load(model_dir / "Y_train.joblib")
Y_test = jb.load(model_dir / "Y_test.joblib")


In [3]:
print("Loaded tuned model:", type(model).__name__)
print("Train shape:", X_train.shape)
print("Test shape :", X_test.shape)

Loaded tuned model: RandomForestRegressor
Train shape: (5922, 52)
Test shape : (1481, 52)


In [6]:
# Top 20 features are
importances = model.feature_importances_
fi = pd.Series(importances, index=features).sort_values(ascending=False)

print("Top 20 features: ")
fi.head(20)

Top 20 features: 


Votes                           0.386205
City_Freq                       0.195373
Average Cost for two            0.108627
Price range                     0.078260
Country Code                    0.073360
Cuisine_Grouped_North Indian    0.026360
Has Online delivery             0.014178
Has Table booking               0.013804
Cuisine_Grouped_Cafe            0.011380
Cuisine_Grouped_Chinese         0.007632
Cuisine_Grouped_Continental     0.007387
Cuisine_Grouped_Other           0.006902
Cuisine_Grouped_Pizza           0.005373
Cuisine_Grouped_Italian         0.005247
Cuisine_Grouped_Asian           0.004168
Cuisine_Grouped_Bakery          0.004137
Cuisine_Grouped_Fast Food       0.004032
Cuisine_Grouped_Desserts        0.003622
Cuisine_Grouped_Ice Cream       0.003250
Cuisine_Grouped_European        0.003005
dtype: float64